In [1]:
import os

In [2]:
%pwd

'/Users/varunsardana/Chest-Classification-using-Mlflow-DVC/research'

In [3]:
os.chdir("../")


In [4]:
%pwd

'/Users/varunsardana/Chest-Classification-using-Mlflow-DVC'

In [1]:
import os
import mlflow

os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/varunsardana2006/Chest-Classification-using-Mlflow-DVC.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'varunsardana2006'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '6c78c96e08f9f18793ebde7944e6b4017fcfa21b'

mlflow.set_experiment("my_experiment")

with mlflow.start_run():
    mlflow.log_param("lr", 0.001)
    mlflow.log_metric("accuracy", 0.95)

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [8]:
from chestcancerClassifier.constants import *
from chestcancerClassifier.utils.common import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:  #the -> EvaluationConfig part tells that it returns an EvaluationConfig object
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/varunsardana2006/Chest-Classification-using-Mlflow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [10]:
import tensorflow as tf
import keras
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [11]:

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        import tensorflow.keras
        tensorflow.keras.__version__ = tf.__version__

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )

            # Always use keras flavor to avoid distutils error
            mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")

In [12]:
import os
import mlflow

# Set environment credentials
os.environ["MLFLOW_TRACKING_USERNAME"] = "varunsardana2006"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "6c78c96e08f9f18793ebde7944e6b4017fcfa21b"

# Set the tracking URI to DagsHub
mlflow.set_tracking_uri("https://dagshub.com/varunsardana2006/Chest-Classification-using-Mlflow-DVC.mlflow")

# Your run ID to delete
run_id = "72f64853dd2b4b389c255c6b0fd754d5"

# Delete the run
mlflow.delete_run(run_id)

print(f"✅ Run {run_id} deleted successfully.")

✅ Run 72f64853dd2b4b389c255c6b0fd754d5 deleted successfully.


In [23]:
import os
import mlflow

# Set environment credentials
os.environ["MLFLOW_TRACKING_USERNAME"] = "varunsardana2006"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "6c78c96e08f9f18793ebde7944e6b4017fcfa21b"

# Set the tracking URI to DagsHub
mlflow.set_tracking_uri("https://dagshub.com/varunsardana2006/Chest-Classification-using-Mlflow-DVC.mlflow")

# Your run ID to delete
run_id = "d2a280fa011d4613b578981918881d6a"

# Delete the run
mlflow.delete_run(run_id)

print(f"✅ Run {run_id} deleted successfully.")

✅ Run d2a280fa011d4613b578981918881d6a deleted successfully.


In [24]:
import os
import mlflow

# Set environment credentials
os.environ["MLFLOW_TRACKING_USERNAME"] = "varunsardana2006"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "6c78c96e08f9f18793ebde7944e6b4017fcfa21b"

# Set the tracking URI to DagsHub
mlflow.set_tracking_uri("https://dagshub.com/varunsardana2006/Chest-Classification-using-Mlflow-DVC.mlflow")

# Your run ID to delete
run_id = "db75c12806f64e019da6f3dc6348d036"

# Delete the run
mlflow.delete_run(run_id)

print(f"✅ Run {run_id} deleted successfully.")

✅ Run db75c12806f64e019da6f3dc6348d036 deleted successfully.


In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-07-30 12:48:08,148: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-07-30 12:48:08,151: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-30 12:48:08,152: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 28s 4s/step - loss: 15.3334 - accuracy: 0.4314
[2025-07-30 12:48:36,649: INFO: common: json file saved at: scores.json]


2025/07/30 12:48:37 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/t2/_lp651dj2b1fcvgx6gpfll6w0000gn/T/tmpqa3sw4pc/model/data/model/assets
[2025-07-30 12:48:38,506: INFO: builder_impl: Assets written to: /var/folders/t2/_lp651dj2b1fcvgx6gpfll6w0000gn/T/tmpqa3sw4pc/model/data/model/assets]


2025/07/30 12:48:44 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/t2/_lp651dj2b1fcvgx6gpfll6w0000gn/T/tmpqa3sw4pc/model, flavor: tensorflow), fall back to return ['tensorflow==2.13.0']. Set logging level to DEBUG to see the full traceback.
/opt/miniconda3/envs/cancer/lib/python3.8/site-packages/_distutils_hack/__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Successfully registered model 'VGG16Model'.
2025/07/30 12:48:56 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
